# Bağlantı

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls

adc.json  drive  sample_data


# Kütüphane

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import string #noktalama işaretleri
import re
import nltk #etkisiz kelimeler
import numpy
import csv
import pandas as pd
import os, glob
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import requests




# Test için örnek dava

In [ ]:

# excel'den veri yükleme

from openpyxl import Workbook,load_workbook 
hukuk = pd.read_excel('/content/drive/Colab Notebooks/Hukuk/hukukk.xlsx')

#ilk beş girdi görmek için
hukuk.head() 

# Veri Seti Önişleme

In [ ]:

nltk.download('stopwords') # önemliiiii
noktalama = string.punctuation
etkisiz = stopwords.words('turkish')
print(noktalama)
print(etkisiz)

# Eğitim Verileri

In [ ]:
dizi= []
for d in hukuk['Karar'].head():

    print(d+ '\n------------------------------')
    # Etkisiz kelimeleri atmak
    temp= ' '
    for word in d.split():
        if word not in etkisiz and not word.isnumeric():
            temp += word + ' '
      
    print(temp+ '\n********************')
    dizi.append(temp)
   

In [ ]:
for d in hukuk['Karar'].head():
    print(d+ '\n------------------------------')
    # Noktalama işaretleri atma
    temp= ''
    for word in d:
        if word not in noktalama:
            temp += word
    print(temp+ '\n********************')
    

# Verileri Kayıt etme

In [ ]:
hukuk.to_csv('/content/drive/Colab Notebooks/Hukuk/cleaned.csv', index = False)

In [ ]:
hukuk = pd.read_csv('/content/drive/Colab Notebooks/Hukuk/cleaned.csv')
print(hukuk.head())

# Veri Setini Bölme

In [ ]:
# Arındırılmış veriyi train ve test kümelerine ayırıyoruz
# Eğitim ve test olarak ayıracağız. train=eğitim, test=test

X_train, X_test, y_train, y_test = train_test_split(hukuk['Karar'].values.astype('U'),hukuk['Hukum'].values.astype('U'), test_size=0.20, random_state=1)

print(X_train.shape)
print(X_test.shape)                                                                      

(43,)
(11,)


# Sayma Vektörü Oluşturma

In [ ]:
# Train kümesindeki cümlelerin sayma vektöelerini çıkarıyoruz
count_vect = CountVectorizer(max_features = 2000000) # Burası önemli ram için
X_train_counts = count_vect.fit_transform(X_train)
print(X_train_counts.shape)

# Tf*Idf vektörü oluşturma sadece o "karara" özgü(temsil edecek kelimler olmalı)

In [ ]:
# Train kümesindeki cümlelerin TF*IDF vektörlerini sayma vekttörlerinden oluşturuyoruz
# Tf= terim sıklığı, IDF= bir kelimenin dökümanda kaç kere geçtiği
# TF*IDF bütün olarak; bir kelimenin bir döküman içinde ki önemini gösterir
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)

# Naive Bayes Model Eğitimi

In [ ]:
# Çok modlu Naive Bayes Sınıflandırıcısı eğitiyoruz
clf = MultinomialNB().fit(X_train_tfidf, y_train)
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

# Model Performansı Ölçme

In [ ]:
# Sınıflandırıcı ile test seti üzerindeki tahminleme yapıyoruz
y_pred = clf.predict(X_test_tfidf)
for review, sentiment in zip(X_test[:], y_pred[:]):
  print('%r => %s' % (review, sentiment))

In [ ]:
# Performans sonuçları
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred))